In [114]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

# 1 Get all players

In [116]:
players = pd.read_csv('https://www.elost.cz/api_1_0/zebricek_oddil/stolni_tenis_benesov', sep=';')['hráč id']
players

0        miffek_daniel_1976
1        lubos_penkava_1965
2      dohnal_jaroslav_1988
3           sturc_jiri_1989
4        martin_dvorak_2005
5         krejci_milan_1968
6            jezo_ivan_1977
7     zvolsky_miroslav_1964
8       martin_melisek_2002
9       zrcek_miroslav_1952
10    nadvornik_stepan_1997
11         roman_zydyk_1996
12    hausenblas_jakub_1997
13           srb_pavel_1978
14         tomas_bidlo_1989
15        povolny_ales_1968
16       hrazdira_jiri_1948
17         kosar_pavel_1952
18          mohr_tomas_1979
19          jan_plisek_2003
20        pavel_plisek_2008
21       ondrej_kohout_1982
22     vladimir_vlasak_1967
23         david_sachl_1972
24          jan_hnatek_1946
25    michael_schuster_1948
26     miroslav_mrazek_1964
Name: hráč id, dtype: object

# 2 Download games for each player and create result dataframe with all games

In [142]:
result = pd.DataFrame()
start = time.time()

for player in players:
    print(player)
    year = int(player[-4:])
    url = f'https://www.elost.cz/st/hrac/{player}?filtr_i_vyhry=ano&filtr_i_prohry=ano&filtr_od_sezony=2011&filtr_zapasy_search=podle+soupe%C5%99e+nebo+odd%C3%ADlu#hrac_zapasy'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    games = str(soup.find_all("table", class_="rank"))
    d = games.replace('<br/>', '|')
    df = pd.read_html(d)[1]
    df[['Hráč', 'Oddíl', 'Družstvo']] = df['Hráč|Oddíl|Družstvo'].str.split('|', expand=True)
    df[['Soupeř', 'Oddíl soupeř', 'Družstvo soupeř']] = df['Soupeř|Oddíl|Družstvo'].str.split('|', expand=True)
    df[['Výsledek', 'Datum zápasu', 'Soutěž']] = df['Výsledek|Datum zápasu|Soutěž'].str.split('|', expand=True)
    df[['Elo hráče', 'Elo (min)', 'Elo (max)']] = df['Elo hráče  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
    df[['Elo soupeře', 'Elo (min) soupeře', 'Elo (max) soupeře']] = df['Elo soupeře  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
    df['Rok narození'] = year
    main_section = soup.find("div", {"id": "main_section"}).text
    try:
        df['Max elo'] = int(re.compile("Nejvyšší dosažené Elo: [0-9]{3,4}").findall(main_section)[0][-4:])
    except:
        df['Max elo'] = None
    try:
        df['Elo nejlepšího poraženého soupeře'] = int(re.compile("Nejlépe hodnocený hráč, kterého .* porazil\(a\): .*, Elo: ([0-9]{3,4})").findall(main_section)[0])
    except:
        df['Elo nejlepšího poraženého soupeře'] = None

    df['ID Hráč'] = player

    df['ID Soupeř'] = re.compile('st/hrac/(.*)"').findall(d)
    df['Rok narození soupeř'] = df['ID Soupeř'].apply(lambda x: int(x[-4:]) if (x[-4:].find("-") <= 0) else int(x[-6:-2]))
    player_games = df.iloc[:, 5:]
    result = result.append(player_games)
result
end = time.time()
print(end - start)

miffek_daniel_1976
lubos_penkava_1965
dohnal_jaroslav_1988
sturc_jiri_1989
martin_dvorak_2005
krejci_milan_1968
jezo_ivan_1977
zvolsky_miroslav_1964
martin_melisek_2002
zrcek_miroslav_1952
nadvornik_stepan_1997
roman_zydyk_1996
hausenblas_jakub_1997
srb_pavel_1978
tomas_bidlo_1989
povolny_ales_1968
hrazdira_jiri_1948
kosar_pavel_1952
mohr_tomas_1979
jan_plisek_2003
pavel_plisek_2008
ondrej_kohout_1982
vladimir_vlasak_1967
david_sachl_1972
jan_hnatek_1946
michael_schuster_1948
miroslav_mrazek_1964
96.66980504989624


In [143]:
result.to_csv('data/1_raw.csv', index=False)

Test ziskani vice udaju o souperi (rok narozeni, max elo a nejlepsi porazeny souper)

In [106]:
# def get_player_info(player):
#     print(player)
#     year = int(player[-4:])
#     url = f'https://www.elost.cz/st/hrac/{player}?filtr_i_vyhry=ano&filtr_i_prohry=ano&filtr_od_sezony=2011&filtr_zapasy_search=podle+soupe%C5%99e+nebo+odd%C3%ADlu#hrac_zapasy'
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     main_section = soup.find("div", {"id": "main_section"}).text
#     max_elo = int(re.compile("Nejvyšší dosažené Elo: [0-9]{3,4}").findall(main_section)[0][-4:])
#     best_souper = int(re.compile("Nejlépe hodnocený hráč, kterého .* porazil\(a\): .*, Elo: ([0-9]{3,4})").findall(main_section)[0])
    
#     return (year, max_elo, best_souper)

In [ ]:
# result = pd.DataFrame()
# player = 'martin_dvorak_2005'
# year = int(player[-4:])
# url = f'https://www.elost.cz/st/hrac/{player}?filtr_i_vyhry=ano&filtr_i_prohry=ano&filtr_od_sezony=2011&filtr_zapasy_search=podle+soupe%C5%99e+nebo+odd%C3%ADlu#hrac_zapasy'
# page = requests.get(url)
# soup = BeautifulSoup(page.content, 'html.parser')
# games = str(soup.find_all("table", class_="rank"))
# d = games.replace('<br/>', '|')
# df = pd.read_html(d)[1]
# df[['Hráč', 'Oddíl', 'Družstvo']] = df['Hráč|Oddíl|Družstvo'].str.split('|', expand=True)
# df[['Soupeř', 'Oddíl soupeř', 'Družstvo soupeř']] = df['Soupeř|Oddíl|Družstvo'].str.split('|', expand=True)
# df[['Výsledek', 'Datum zápasu', 'Soutěž']] = df['Výsledek|Datum zápasu|Soutěž'].str.split('|', expand=True)
# df[['Elo hráče', 'Elo (min)', 'Elo (max)']] = df['Elo hráče  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
# df[['Elo soupeře', 'Elo (min) soupeře', 'Elo (max) soupeře']] = df['Elo soupeře  |  Elo(min)|  Elo(max)|'].str.split('|', expand=True)
# df['Rok narození'] = year
# main_section = soup.find("div", {"id": "main_section"}).text
# try:
#     df['Max elo'] = int(re.compile("Nejvyšší dosažené Elo: [0-9]{3,4}").findall(main_section)[0][-4:])
# except:
#     df['Max elo'] = None
# try:
#     df['Elo nejlepšího poraženého soupeře'] = int(re.compile("Nejlépe hodnocený hráč, kterého .* porazil\(a\): .*, Elo: ([0-9]{3,4})").findall(main_section)[0])
# except:
#     df['Elo nejlepšího poraženého soupeře'] = None

# df['ID Hráč'] = player

# df['ID Soupeř'] = re.compile('st/hrac/(.*)"').findall(d)
# df['Rok narození soupeř'] = df['ID Soupeř'].apply(lambda x: int(x[-4:]) if (x[-4:].find("-") <= 0) else int(x[-6:-2]))
# # df[['Rok narození soupeř', 'Max elo soupeř', 'Elo nejlepšího poraženého soupeře soupeř']] = df['ID Soupeř'].apply(lambda x: get_player_info(x))
# player_games = df.iloc[:, 5:]
# player_games

Bohuzel ziskani informaci o jednom souperi trvalo prilis dlouho, proto o souperi bude ziskan pouze rok narozeni (z ID hrace)